# Neural Network Initialization and Training Diagnostics: A Step-by-Step Tutorial

This tutorial will guide you through understanding:
- Why proper initialization matters
- How to diagnose activation and gradient problems
- What batch normalization does and how it helps
- How to build modular neural networks

**Learning Approach**: Each section follows the pattern:
1. **Motivation** - Why does this matter?
2. **Exercise** - Try it yourself
3. **Solution** - Check your understanding

Let's start from the basics and build up!

## Setup: Loading the Data

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Read in all the words
words = open('names.txt', 'r').read().splitlines()
print(f"Total words: {len(words)}")
print(f"First 8 words: {words[:8]}")

In [ ]:
# Build vocabulary
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(f"Vocabulary size: {vocab_size}")
print(f"Characters: {''.join(itos[i] for i in range(1, vocab_size))}")

In [ ]:
# Build dataset
block_size = 3  # context length

def build_dataset(words):  
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

print(f"Training set: {Xtr.shape[0]} examples")
print(f"Dev set: {Xdev.shape[0]} examples")
print(f"Test set: {Xte.shape[0]} examples")

---
## Part 1: Understanding Loss at Initialization

### 🎯 Motivation

When we start training a neural network, we should have a rough idea of what loss to expect. This is like checking if your car starts before driving - if something is wrong at the start, it will cause problems later.

For a character-level language model:
- We have 27 possible characters (a-z + '.')
- At initialization, the network knows nothing
- It should predict each character with equal probability: 1/27
- The loss (negative log likelihood) should be: -log(1/27) ≈ 3.29

**If your initial loss is much higher (like 27!), something is very wrong.**

### 📝 Exercise 1.1: Calculate Expected Initial Loss

**Task**: Calculate what the loss should be at initialization if the model predicts uniformly.

Hints:
- Probability for any character: 1/27
- Loss = -log(probability)
- Use `torch.tensor()` and `.log()` for calculations

In [ ]:
# YOUR CODE HERE
# Calculate the expected loss at initialization
prob = # ?
expected_loss = # ?
print(f"Expected loss at initialization: {expected_loss:.4f}")

### ✅ Solution 1.1

In [ ]:
# SOLUTION
prob = torch.tensor(1/27)
expected_loss = -prob.log()
print(f"Expected loss at initialization: {expected_loss:.4f}")
print(f"\nExplanation: If the model assigns equal probability (1/27 = 0.037) to each character,")
print(f"then the negative log probability is {expected_loss:.4f}")

### 📝 Exercise 1.2: Create a Poorly Initialized Network

**Task**: Create a simple MLP and observe what happens with naive initialization.

Network architecture:
- Embedding: 27 chars → 10 dimensions
- Hidden layer: 30 inputs (10 * 3 context) → 200 neurons
- Output layer: 200 → 27 logits

Use `torch.randn()` for all weights/biases (this is WRONG on purpose!)

In [ ]:
# YOUR CODE HERE
n_embd = 10
n_hidden = 200
g = torch.Generator().manual_seed(2147483647)

# Initialize parameters (poorly!)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = # ?
b1 = # ?
W2 = # ?
b2 = # ?

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

print(f"Total parameters: {sum(p.nelement() for p in parameters)}")

### ✅ Solution 1.2

In [ ]:
# SOLUTION
n_embd = 10
n_hidden = 200
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)
W2 = torch.randn((n_hidden, vocab_size), generator=g)
b2 = torch.randn(vocab_size, generator=g)

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

print(f"Total parameters: {sum(p.nelement() for p in parameters)}")

### 📝 Exercise 1.3: Observe the Initial Loss Problem

**Task**: Do a forward pass on a single batch and check the initial loss.

In [ ]:
# YOUR CODE HERE
batch_size = 32
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

# Forward pass
emb = # ?
embcat = # ?
h = # ?
logits = # ?
loss = # ?

print(f"Initial loss: {loss.item():.4f}")
print(f"Expected loss: ~3.29")
print(f"\nProblem: Loss is way too {'high' if loss > 10 else 'close'}!")

### ✅ Solution 1.3

In [ ]:
# SOLUTION
batch_size = 32
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h = torch.tanh(embcat @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yb)

print(f"Initial loss: {loss.item():.4f}")
print(f"Expected loss: ~3.29")
print(f"\n🔍 Analysis:")
print(f"Sample logits: {logits[0, :5]}")
print(f"\nThe logits have extreme values! This makes the softmax very confident (and wrong).")
print(f"This is like a student who confidently gives wrong answers on every question.")

### 📝 Exercise 1.4: Fix the Output Layer

**Task**: Fix the initialization of W2 and b2 so logits are close to zero at initialization.

Hints:
- Multiply W2 by a small number (try 0.01)
- Set b2 to zeros

In [ ]:
# YOUR CODE HERE
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)
W2 = # ? Fix this
b2 = # ? Fix this

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

# Test it
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h = torch.tanh(embcat @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yb)

print(f"Initial loss: {loss.item():.4f}")
print(f"Expected: ~3.29")

### ✅ Solution 1.4

In [ ]:
# SOLUTION
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01  # Scale down!
b2 = torch.zeros(vocab_size)  # No bias needed

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h = torch.tanh(embcat @ W1 + b1)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yb)

print(f"Initial loss: {loss.item():.4f}")
print(f"Expected: ~3.29")
print(f"\n✅ Much better! Now the network starts from a reasonable place.")
print(f"Sample logits: {logits[0, :5]}")
print(f"These are close to zero, so softmax gives ~uniform probabilities.")

---
## Part 2: Understanding Tanh Saturation

### 🎯 Motivation

The tanh activation function squashes inputs to the range [-1, 1].

**The Problem:**
- If inputs to tanh are very large (positive or negative), outputs get stuck at ±1
- This is called "saturation"
- Saturated neurons have zero gradient → they don't learn!

**Why it matters:**
- Gradient of tanh: (1 - tanh²(x))
- If tanh(x) = 1, then gradient = 0
- If tanh(x) = -1, then gradient = 0
- No gradient = no learning = "dead neuron"

### 📝 Exercise 2.1: Visualize Tanh and Its Gradient

**Task**: Plot tanh(x) and its gradient to understand saturation.

In [ ]:
# YOUR CODE HERE
import numpy as np

x = torch.linspace(-5, 5, 100)
y = # ? Apply tanh
grad = # ? Calculate gradient: 1 - y^2

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
# Plot tanh
plt.subplot(1, 2, 2)
# Plot gradient
plt.show()

### ✅ Solution 2.1

In [ ]:
# SOLUTION
x = torch.linspace(-5, 5, 100)
y = torch.tanh(x)
grad = 1 - y**2

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(x, y)
plt.axhline(y=1, color='r', linestyle='--', alpha=0.3)
plt.axhline(y=-1, color='r', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.title('tanh(x) - The Activation')
plt.xlabel('x')
plt.ylabel('tanh(x)')

plt.subplot(1, 2, 2)
plt.plot(x, grad)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.title('Gradient: 1 - tanh²(x)')
plt.xlabel('x')
plt.ylabel('gradient')

plt.tight_layout()
plt.show()

print("🔍 Key Observations:")
print("1. When |x| > 3, tanh(x) ≈ ±1 (saturated)")
print("2. When saturated, gradient ≈ 0 (no learning!)")
print("3. Maximum gradient is at x=0 (gradient = 1)")

### 📝 Exercise 2.2: Check Tanh Saturation in Our Network

**Task**: Calculate the pre-activations (before tanh) and check how many neurons are saturated.

A neuron is "saturated" if |tanh(x)| > 0.97

In [ ]:
# YOUR CODE HERE
# Use the poorly initialized network from before
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h_preact = # ? Calculate pre-activation
h = # ? Apply tanh

# Visualize
plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.hist(h_preact.detach().flatten().numpy(), bins=50)
plt.title('Pre-activation distribution')
plt.xlabel('Value')

plt.subplot(1, 2, 2)
plt.hist(h.detach().flatten().numpy(), bins=50)
plt.title('Post-activation (tanh) distribution')
plt.xlabel('Value')
plt.show()

# Calculate saturation
saturated = # ?
print(f"Percentage of saturated neurons: {saturated * 100:.2f}%")

### ✅ Solution 2.2

In [ ]:
# SOLUTION
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g)
b1 = torch.randn(n_hidden, generator=g)

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h_preact = embcat @ W1 + b1
h = torch.tanh(h_preact)

plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.hist(h_preact.detach().flatten().numpy(), bins=50)
plt.title('Pre-activation distribution')
plt.xlabel('Value')
plt.axvline(x=-3, color='r', linestyle='--', label='Saturation zone')
plt.axvline(x=3, color='r', linestyle='--')
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(h.detach().flatten().numpy(), bins=50, range=(-1, 1))
plt.title('Post-activation (tanh) distribution')
plt.xlabel('Value')
plt.axvline(x=-0.97, color='r', linestyle='--', label='Saturated')
plt.axvline(x=0.97, color='r', linestyle='--')
plt.legend()
plt.tight_layout()
plt.show()

saturated = (h.abs() > 0.97).float().mean()
print(f"Percentage of saturated neurons: {saturated * 100:.2f}%")
print(f"\n⚠️ Problem: Many neurons are saturated!")
print(f"Pre-activations range: [{h_preact.min():.2f}, {h_preact.max():.2f}]")
print(f"These are too extreme → tanh saturates → gradients vanish")

### 📝 Exercise 2.3: Fix Tanh Saturation

**Task**: Scale down W1 and b1 so pre-activations are closer to zero.

Try multiplying W1 and b1 by 0.2 (or experiment with other values).

In [ ]:
# YOUR CODE HERE
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = # ? Fix this
b1 = # ? Fix this

# Test it
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h_preact = embcat @ W1 + b1
h = torch.tanh(h_preact)

saturated = (h.abs() > 0.97).float().mean()
print(f"Percentage of saturated neurons: {saturated * 100:.2f}%")
print(f"Pre-activations range: [{h_preact.min():.2f}, {h_preact.max():.2f}]")

### ✅ Solution 2.3

In [ ]:
# SOLUTION
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * 0.2
b1 = torch.randn(n_hidden, generator=g) * 0.01

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h_preact = embcat @ W1 + b1
h = torch.tanh(h_preact)

plt.figure(figsize=(14, 4))
plt.subplot(1, 2, 1)
plt.hist(h_preact.detach().flatten().numpy(), bins=50)
plt.title('Pre-activation distribution (Fixed!)')
plt.xlabel('Value')

plt.subplot(1, 2, 2)
plt.hist(h.detach().flatten().numpy(), bins=50, range=(-1, 1))
plt.title('Post-activation distribution (Fixed!)')
plt.xlabel('Value')
plt.tight_layout()
plt.show()

saturated = (h.abs() > 0.97).float().mean()
print(f"Percentage of saturated neurons: {saturated * 100:.2f}%")
print(f"Pre-activations range: [{h_preact.min():.2f}, {h_preact.max():.2f}]")
print(f"\n✅ Much better! Neurons are now active and can learn.")

---
## Part 3: Kaiming Initialization

### 🎯 Motivation

We've been guessing numbers (0.2, 0.01, etc.) to fix initialization. This is not scalable!

**Kaiming Initialization** (He et al., 2015) provides a principled way to initialize weights:
- Goal: Keep activations at unit Gaussian (mean=0, std=1) throughout the network
- Key insight: If inputs have std=1, outputs will too if we scale weights by 1/√(fan_in)

**For different activations:**
- Linear (no activation): gain = 1.0
- ReLU: gain = √2 (because ReLU kills half the neurons)
- Tanh: gain = 5/3 (because tanh is contractive)

### 📝 Exercise 3.1: Understand Fan-in Normalization

**Task**: Demonstrate why we need to divide by √(fan_in)

Create:
1. Random input x with 1000 examples, 10 features (mean=0, std=1)
2. Random weight W with 10 inputs, 200 outputs
3. Calculate y = x @ W
4. Check the std of y with different weight scalings

In [ ]:
# YOUR CODE HERE
fan_in = 10
fan_out = 200

x = torch.randn(1000, fan_in)
print(f"Input x: mean={x.mean():.4f}, std={x.std():.4f}")

# Test 1: No scaling
W = torch.randn(fan_in, fan_out)
y = x @ W
print(f"\nNo scaling: output std = {y.std():.4f}")

# Test 2: Scale by fan_in
W = torch.randn(fan_in, fan_out) / fan_in
y = x @ W
print(f"Scale by fan_in: output std = {y.std():.4f}")

# Test 3: Scale by sqrt(fan_in) - CORRECT!
W = # ?
y = # ?
print(f"Scale by sqrt(fan_in): output std = {y.std():.4f} ← Should be ~1.0!")

### ✅ Solution 3.1

In [ ]:
# SOLUTION
fan_in = 10
fan_out = 200

x = torch.randn(1000, fan_in)
print(f"Input x: mean={x.mean():.4f}, std={x.std():.4f}")

# Test 1: No scaling
W = torch.randn(fan_in, fan_out)
y = x @ W
print(f"\nNo scaling: output std = {y.std():.4f} ❌ (too large!)")

# Test 2: Scale by fan_in
W = torch.randn(fan_in, fan_out) / fan_in
y = x @ W
print(f"Scale by fan_in: output std = {y.std():.4f} ❌ (too small!)")

# Test 3: Scale by sqrt(fan_in) - CORRECT!
W = torch.randn(fan_in, fan_out) / fan_in**0.5
y = x @ W
print(f"Scale by sqrt(fan_in): output std = {y.std():.4f} ✅ (perfect!)")

print(f"\n📊 Explanation:")
print(f"When we multiply x by W, we're summing {fan_in} random products.")
print(f"Variance adds up: var(sum) = sum(var) = {fan_in} * var")
print(f"So std grows by sqrt({fan_in}) = {fan_in**0.5:.2f}")
print(f"To compensate, divide by sqrt({fan_in})!")

### 📝 Exercise 3.2: Apply Kaiming Initialization

**Task**: Initialize the network using Kaiming initialization with the correct gain for tanh.

In [ ]:
# YOUR CODE HERE
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)

fan_in = n_embd * block_size  # = 30
gain = # ? (for tanh, gain = 5/3)
W1 = # ? Apply Kaiming init
b1 = # ? Small or zero

W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01
b2 = torch.zeros(vocab_size)

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

# Test
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h_preact = embcat @ W1 + b1
h = torch.tanh(h_preact)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yb)

print(f"Pre-activation std: {h_preact.std():.4f} (should be ~1.0)")
print(f"Activation std: {h.std():.4f} (should be ~0.6-0.7 for tanh)")
print(f"Saturation: {(h.abs() > 0.97).float().mean() * 100:.2f}%")
print(f"Initial loss: {loss.item():.4f} (should be ~3.29)")

### ✅ Solution 3.2

In [ ]:
# SOLUTION
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)

fan_in = n_embd * block_size
gain = 5/3  # For tanh
W1 = torch.randn((fan_in, n_hidden), generator=g) * gain / (fan_in ** 0.5)
b1 = torch.zeros(n_hidden)  # Can also use small random values

W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01
b2 = torch.zeros(vocab_size)

parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h_preact = embcat @ W1 + b1
h = torch.tanh(h_preact)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yb)

print(f"Pre-activation std: {h_preact.std():.4f} (should be ~1.0) ✅")
print(f"Activation std: {h.std():.4f} (should be ~0.6-0.7 for tanh) ✅")
print(f"Saturation: {(h.abs() > 0.97).float().mean() * 100:.2f}% ✅")
print(f"Initial loss: {loss.item():.4f} (should be ~3.29) ✅")

print(f"\n🎉 Perfect initialization! The network is ready to train.")
print(f"\nFormula: W = torch.randn(...) * (gain / sqrt(fan_in))")
print(f"Where: fan_in = {fan_in}, gain = {gain:.4f}")

---
## Part 4: Batch Normalization

### 🎯 Motivation

Even with Kaiming initialization, deep networks are hard to train. Activations can still drift during training.

**Batch Normalization** (2015): If we want activations to be Gaussian, why not just... make them Gaussian?

**The trick:**
1. Normalize: Make activations mean=0, std=1
2. Scale & Shift: Let the network learn the best distribution via γ (gain) and β (bias)

**Benefits:**
- Stabilizes training
- Allows higher learning rates
- Reduces dependence on initialization

**Downside:**
- Couples examples in a batch (weird!)
- Different behavior in train vs eval mode
- Can be source of bugs

### 📝 Exercise 4.1: Implement Basic Batch Normalization

**Task**: Implement the forward pass of batch normalization.

Steps:
1. Calculate mean and std across the batch dimension
2. Normalize: (x - mean) / std
3. Scale and shift: γ * x_normalized + β

In [ ]:
# YOUR CODE HERE
# Create some random pre-activations
h_preact = torch.randn(32, 200) * 3  # 32 examples, 200 neurons, large values

print(f"Before BatchNorm:")
print(f"Mean: {h_preact.mean(0)[:5]}")
print(f"Std: {h_preact.std(0)[:5]}")

# BatchNorm parameters
bn_gain = torch.ones((1, 200))
bn_bias = torch.zeros((1, 200))

# TODO: Implement BatchNorm
bn_mean = # ? Calculate mean
bn_std = # ? Calculate std
h_normalized = # ? Normalize
h_bn = # ? Scale and shift

print(f"\nAfter BatchNorm:")
print(f"Mean: {h_bn.mean(0)[:5]}")
print(f"Std: {h_bn.std(0)[:5]}")

### ✅ Solution 4.1

In [ ]:
# SOLUTION
h_preact = torch.randn(32, 200) * 3

print(f"Before BatchNorm:")
print(f"Mean: {h_preact.mean(0)[:5]}")
print(f"Std: {h_preact.std(0)[:5]}")

bn_gain = torch.ones((1, 200))
bn_bias = torch.zeros((1, 200))

# BatchNorm forward pass
bn_mean = h_preact.mean(0, keepdim=True)
bn_std = h_preact.std(0, keepdim=True)
h_normalized = (h_preact - bn_mean) / bn_std
h_bn = bn_gain * h_normalized + bn_bias

print(f"\nAfter BatchNorm:")
print(f"Mean: {h_bn.mean(0)[:5]} ← Should be ~0")
print(f"Std: {h_bn.std(0)[:5]} ← Should be ~1")

print(f"\n✅ Each neuron now has mean≈0, std≈1!")
print(f"\n🔑 Key insight: We normalized across the BATCH dimension (dim=0)")
print(f"   Shape: (32 examples, 200 neurons)")
print(f"   For each neuron, we computed mean/std over all 32 examples")

### 📝 Exercise 4.2: Add BatchNorm to the Network

**Task**: Integrate batch normalization into the MLP.

In [ ]:
# YOUR CODE HERE
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) / (n_embd * block_size)**0.5
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01
b2 = torch.zeros(vocab_size)

# BatchNorm parameters
bn_gain = # ?
bn_bias = # ?

parameters = [C, W1, W2, b2, bn_gain, bn_bias]
for p in parameters:
    p.requires_grad = True

print(f"Total parameters: {sum(p.nelement() for p in parameters)}")

# Forward pass with BatchNorm
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h_preact = embcat @ W1

# TODO: Apply BatchNorm here
bn_mean = # ?
bn_std = # ?
h_preact = # ? Normalize, scale, shift

h = torch.tanh(h_preact)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yb)

print(f"\nInitial loss: {loss.item():.4f}")
print(f"Saturation: {(h.abs() > 0.97).float().mean() * 100:.2f}%")

### ✅ Solution 4.2

In [ ]:
# SOLUTION
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) / (n_embd * block_size)**0.5
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01
b2 = torch.zeros(vocab_size)

bn_gain = torch.ones((1, n_hidden))
bn_bias = torch.zeros((1, n_hidden))

parameters = [C, W1, W2, b2, bn_gain, bn_bias]
for p in parameters:
    p.requires_grad = True

print(f"Total parameters: {sum(p.nelement() for p in parameters)}")

ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)
h_preact = embcat @ W1

# BatchNorm layer
bn_mean = h_preact.mean(0, keepdim=True)
bn_std = h_preact.std(0, keepdim=True)
h_preact = bn_gain * (h_preact - bn_mean) / bn_std + bn_bias

h = torch.tanh(h_preact)
logits = h @ W2 + b2
loss = F.cross_entropy(logits, Yb)

print(f"\nInitial loss: {loss.item():.4f} ✅")
print(f"Saturation: {(h.abs() > 0.97).float().mean() * 100:.2f}% ✅")
print(f"\n🎉 BatchNorm ensures good activations regardless of W1 initialization!")

### 📝 Exercise 4.3: Running Statistics for Inference

**Task**: During training, we normalize using batch statistics. But at inference, we might have just one example!

Solution: Keep running estimates of mean and std during training.

Implement the running mean update: `running = 0.999 * running + 0.001 * current`

In [ ]:
# YOUR CODE HERE
g = torch.Generator().manual_seed(2147483647)

# Initialize network (same as before)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) / (n_embd * block_size)**0.5
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01
b2 = torch.zeros(vocab_size)
bn_gain = torch.ones((1, n_hidden))
bn_bias = torch.zeros((1, n_hidden))

# Running statistics (buffers, not parameters!)
bn_mean_running = # ?
bn_std_running = # ?

parameters = [C, W1, W2, b2, bn_gain, bn_bias]
for p in parameters:
    p.requires_grad = True

# Simulate training for a few steps
for i in range(100):
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]
    
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)
    h_preact = embcat @ W1
    
    # Batch statistics
    bn_mean_i = h_preact.mean(0, keepdim=True)
    bn_std_i = h_preact.std(0, keepdim=True)
    
    # TODO: Update running statistics
    with torch.no_grad():
        bn_mean_running = # ?
        bn_std_running = # ?
    
    # Use batch statistics for this forward pass
    h_preact = bn_gain * (h_preact - bn_mean_i) / bn_std_i + bn_bias
    h = torch.tanh(h_preact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb)

print(f"Running mean (first 5): {bn_mean_running[0, :5]}")
print(f"Running std (first 5): {bn_std_running[0, :5]}")
print(f"\nThese will be used at inference time!")

### ✅ Solution 4.3

In [ ]:
# SOLUTION
g = torch.Generator().manual_seed(2147483647)

C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) / (n_embd * block_size)**0.5
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01
b2 = torch.zeros(vocab_size)
bn_gain = torch.ones((1, n_hidden))
bn_bias = torch.zeros((1, n_hidden))

bn_mean_running = torch.zeros((1, n_hidden))
bn_std_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bn_gain, bn_bias]
for p in parameters:
    p.requires_grad = True

momentum = 0.001
for i in range(1000):
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]
    
    emb = C[Xb]
    embcat = emb.view(emb.shape[0], -1)
    h_preact = embcat @ W1
    
    bn_mean_i = h_preact.mean(0, keepdim=True)
    bn_std_i = h_preact.std(0, keepdim=True)
    
    # Update running statistics (exponential moving average)
    with torch.no_grad():
        bn_mean_running = (1 - momentum) * bn_mean_running + momentum * bn_mean_i
        bn_std_running = (1 - momentum) * bn_std_running + momentum * bn_std_i
    
    h_preact = bn_gain * (h_preact - bn_mean_i) / bn_std_i + bn_bias
    h = torch.tanh(h_preact)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Yb)

print(f"Running mean (first 5): {bn_mean_running[0, :5]}")
print(f"Running std (first 5): {bn_std_running[0, :5]}")
print(f"\n✅ These running statistics approximate the true mean/std over the entire dataset.")
print(f"At inference, use these instead of batch statistics!")
print(f"\n🔑 Key: with torch.no_grad() prevents building gradients for this update")

---
## Part 5: Building Modular Networks (PyTorch Style)

### 🎯 Motivation

Writing neural networks with explicit matrix multiplications gets messy. Let's create reusable modules!

**We'll build:**
1. `Linear` - A linear layer (like `nn.Linear`)
2. `BatchNorm1d` - Batch normalization (like `nn.BatchNorm1d`)
3. `Tanh` - Tanh activation (like `nn.Tanh`)

Then stack them like Lego blocks!

### 📝 Exercise 5.1: Create a Linear Layer Class

**Task**: Implement a `Linear` class that:
- Takes fan_in and fan_out in `__init__`
- Initializes weights using Kaiming init
- Implements forward pass in `__call__`
- Returns parameters in `parameters()`

In [ ]:
# YOUR CODE HERE
class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        # TODO: Initialize weight and bias
        self.weight = # ?
        self.bias = # ?
    
    def __call__(self, x):
        # TODO: Implement forward pass
        self.out = # ?
        return self.out
    
    def parameters(self):
        # TODO: Return list of parameters
        return # ?

# Test it
layer = Linear(10, 20)
x = torch.randn(5, 10)
y = layer(x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {y.shape}")
print(f"Number of parameters: {sum(p.nelement() for p in layer.parameters())}")

### ✅ Solution 5.1

In [ ]:
# SOLUTION
class Linear:
    def __init__(self, fan_in, fan_out, bias=True):
        self.weight = torch.randn((fan_in, fan_out)) / fan_in**0.5
        self.bias = torch.zeros(fan_out) if bias else None
    
    def __call__(self, x):
        self.out = x @ self.weight
        if self.bias is not None:
            self.out += self.bias
        return self.out
    
    def parameters(self):
        return [self.weight] + ([] if self.bias is None else [self.bias])

# Test
layer = Linear(10, 20)
x = torch.randn(5, 10)
y = layer(x)
print(f"Input shape: {x.shape}")
print(f"Output shape: {y.shape}")
print(f"Number of parameters: {sum(p.nelement() for p in layer.parameters())}")
print(f"Expected: 10*20 + 20 = {10*20 + 20} ✅")

### 📝 Exercise 5.2: Create BatchNorm1d Class

**Task**: Implement `BatchNorm1d` with training and eval modes.

In [ ]:
# YOUR CODE HERE
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        
        # TODO: Initialize parameters and buffers
        self.gamma = # ? (trainable)
        self.beta = # ? (trainable)
        self.running_mean = # ? (buffer)
        self.running_var = # ? (buffer)
    
    def __call__(self, x):
        # TODO: Implement forward pass
        if self.training:
            xmean = # ?
            xvar = # ?
        else:
            xmean = # ?
            xvar = # ?
        
        xhat = # ? Normalize
        self.out = # ? Scale and shift
        
        # TODO: Update running statistics
        if self.training:
            with torch.no_grad():
                self.running_mean = # ?
                self.running_var = # ?
        
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

# Test it
bn = BatchNorm1d(10)
x = torch.randn(32, 10) * 3
y = bn(x)
print(f"Input mean: {x.mean(0)[0]:.4f}, std: {x.std(0)[0]:.4f}")
print(f"Output mean: {y.mean(0)[0]:.4f}, std: {y.std(0)[0]:.4f}")

### ✅ Solution 5.2

In [ ]:
# SOLUTION
class BatchNorm1d:
    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)
    
    def __call__(self, x):
        if self.training:
            xmean = x.mean(0, keepdim=True)
            xvar = x.var(0, keepdim=True)
        else:
            xmean = self.running_mean
            xvar = self.running_var
        
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps)
        self.out = self.gamma * xhat + self.beta
        
        if self.training:
            with torch.no_grad():
                self.running_mean = (1 - self.momentum) * self.running_mean + self.momentum * xmean
                self.running_var = (1 - self.momentum) * self.running_var + self.momentum * xvar
        
        return self.out
    
    def parameters(self):
        return [self.gamma, self.beta]

bn = BatchNorm1d(10)
x = torch.randn(32, 10) * 3
y = bn(x)
print(f"Input mean: {x.mean(0)[0]:.4f}, std: {x.std(0)[0]:.4f}")
print(f"Output mean: {y.mean(0)[0]:.4f}, std: {y.std(0)[0]:.4f}")
print(f"\n✅ Output is normalized!")

### 📝 Exercise 5.3: Build a Deep Network

**Task**: Stack multiple layers to create a deep network.

Architecture:
- Linear(30, 100) → BatchNorm → Tanh
- Linear(100, 100) → BatchNorm → Tanh
- Linear(100, 27)

In [ ]:
# YOUR CODE HERE
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)

layers = [
    # TODO: Build the layer stack
]

# Initialize last layer to be less confident
with torch.no_grad():
    layers[-1].weight *= 0.1

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(f"Total parameters: {sum(p.nelement() for p in parameters)}")

for p in parameters:
    p.requires_grad = True

# Test forward pass
ix = torch.randint(0, Xtr.shape[0], (32,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
x = emb.view(emb.shape[0], -1)
for layer in layers:
    x = layer(x)
loss = F.cross_entropy(x, Yb)

print(f"Initial loss: {loss.item():.4f}")

### ✅ Solution 5.3

In [ ]:
# SOLUTION
class Tanh:
    def __call__(self, x):
        self.out = torch.tanh(x)
        return self.out
    def parameters(self):
        return []

g = torch.Generator().manual_seed(2147483647)
n_embd = 10
n_hidden = 100

C = torch.randn((vocab_size, n_embd), generator=g)

layers = [
    Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]

with torch.no_grad():
    layers[-1].gamma *= 0.1

parameters = [C] + [p for layer in layers for p in layer.parameters()]
print(f"Total parameters: {sum(p.nelement() for p in parameters)}")

for p in parameters:
    p.requires_grad = True

ix = torch.randint(0, Xtr.shape[0], (32,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
x = emb.view(emb.shape[0], -1)
for layer in layers:
    x = layer(x)
loss = F.cross_entropy(x, Yb)

print(f"Initial loss: {loss.item():.4f}")
print(f"\n✅ We've built a modular, deep neural network!")
print(f"🎉 This is how PyTorch works under the hood!")

---
## Part 6: Diagnostic Visualizations

### 🎯 Motivation

How do you know if your network is training well? Look at the statistics!

**What to monitor:**
1. **Activation distributions** - Are neurons saturated?
2. **Gradient distributions** - Are gradients flowing?
3. **Update-to-parameter ratios** - Is learning rate correct?

**Rules of thumb:**
- Activations should have std ≈ 0.5-1.0
- Gradients should be roughly equal across layers
- Update/parameter ratio should be ≈ 10⁻³

### 📝 Exercise 6.1: Visualize Activation Distributions

**Task**: Plot histograms of activations after each Tanh layer.

In [ ]:
# YOUR CODE HERE
# Do a forward pass
ix = torch.randint(0, Xtr.shape[0], (32,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
x = emb.view(emb.shape[0], -1)
for layer in layers:
    x = layer(x)

# TODO: Create visualization
plt.figure(figsize=(20, 4))
legends = []
for i, layer in enumerate(layers):
    if isinstance(layer, Tanh):
        t = layer.out
        # Calculate stats
        mean = # ?
        std = # ?
        saturated = # ? (|t| > 0.97)
        
        print(f"Layer {i}: mean={mean:.2f}, std={std:.2f}, saturated={saturated*100:.2f}%")
        
        # Plot histogram
        # ?

plt.legend(legends)
plt.title('Activation Distribution')
plt.show()

### ✅ Solution 6.1

In [ ]:
# SOLUTION
ix = torch.randint(0, Xtr.shape[0], (32,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

emb = C[Xb]
x = emb.view(emb.shape[0], -1)
for layer in layers:
    x = layer(x)

plt.figure(figsize=(20, 4))
legends = []
for i, layer in enumerate(layers):
    if isinstance(layer, Tanh):
        t = layer.out
        mean = t.mean()
        std = t.std()
        saturated = (t.abs() > 0.97).float().mean()
        
        print(f"Layer {i} ({layer.__class__.__name__}): mean={mean:.2f}, std={std:.2f}, saturated={saturated*100:.2f}%")
        
        hy, hx = torch.histogram(t.detach(), bins=50, density=True)
        plt.plot(hx[:-1], hy)
        legends.append(f"Layer {i}")

plt.legend(legends)
plt.title('Activation Distribution')
plt.xlabel('Activation value')
plt.ylabel('Density')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\n✅ All layers have similar, healthy distributions!")
print(f"This is thanks to BatchNorm keeping activations normalized.")

### 📝 Exercise 6.2: Monitor Training

**Task**: Train for a few steps and track update-to-parameter ratios.

In [ ]:
# YOUR CODE HERE
max_steps = 1000
batch_size = 32
lossi = []
ud = []  # update to data ratios

for i in range(max_steps):
    # Minibatch
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]
    
    # Forward pass
    emb = C[Xb]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)
    
    # Backward pass
    for p in parameters:
        p.grad = None
    loss.backward()
    
    # Update
    lr = 0.1
    for p in parameters:
        p.data += -lr * p.grad
    
    # Track stats
    lossi.append(loss.log10().item())
    with torch.no_grad():
        # TODO: Calculate update to data ratio
        ud.append([((lr * p.grad).std() / p.data.std()).log10().item() for p in parameters])

# Plot
plt.figure(figsize=(20, 4))
plt.subplot(1, 2, 1)
plt.plot(lossi)
plt.title('Training Loss')

plt.subplot(1, 2, 2)
for i, p in enumerate(parameters):
    if p.ndim == 2:
        plt.plot([ud[j][i] for j in range(len(ud))])
plt.plot([0, len(ud)], [-3, -3], 'k', label='target: 10⁻³')
plt.legend()
plt.title('Update/Parameter Ratio')
plt.show()

### ✅ Solution 6.2

In [ ]:
# SOLUTION
# Reinitialize to start fresh
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
layers = [
    Linear(n_embd * block_size, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, n_hidden, bias=False), BatchNorm1d(n_hidden), Tanh(),
    Linear(n_hidden, vocab_size, bias=False), BatchNorm1d(vocab_size),
]
with torch.no_grad():
    layers[-1].gamma *= 0.1

parameters = [C] + [p for layer in layers for p in layer.parameters()]
for p in parameters:
    p.requires_grad = True

max_steps = 1000
batch_size = 32
lossi = []
ud = []

for i in range(max_steps):
    ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtr[ix], Ytr[ix]
    
    emb = C[Xb]
    x = emb.view(emb.shape[0], -1)
    for layer in layers:
        x = layer(x)
    loss = F.cross_entropy(x, Yb)
    
    for p in parameters:
        p.grad = None
    loss.backward()
    
    lr = 0.1
    for p in parameters:
        p.data += -lr * p.grad
    
    lossi.append(loss.log10().item())
    with torch.no_grad():
        ud.append([((lr * p.grad).std() / p.data.std()).log10().item() for p in parameters])

plt.figure(figsize=(20, 4))
plt.subplot(1, 2, 1)
plt.plot(lossi)
plt.title('Training Loss (log scale)')
plt.xlabel('Step')
plt.ylabel('log10(loss)')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
legends = []
for i, p in enumerate(parameters):
    if p.ndim == 2:
        plt.plot([ud[j][i] for j in range(len(ud))])
        legends.append(f'param {i}')
plt.plot([0, len(ud)], [-3, -3], 'k', linewidth=2)
legends.append('target: 10⁻³')
plt.legend(legends)
plt.title('Update/Parameter Ratio (log scale)')
plt.xlabel('Step')
plt.ylabel('log10(update/param)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n✅ Key observations:")
print(f"1. Loss is decreasing smoothly (learning is happening!)")
print(f"2. Update ratios are near 10⁻³ (learning rate is good!)")
print(f"3. All layers updating at similar rates (balanced training!)")

---
## 🎓 Final Summary

### What You've Learned

**1. Initialization Matters**
- Initial loss should match theoretical expectation
- Bad init → wasted training time or failure to learn
- Use Kaiming init: `weight = randn(...) * gain / sqrt(fan_in)`

**2. Activation Statistics**
- Monitor saturation (too many values at extremes)
- Keep activations in "active" range of nonlinearity
- Tanh: keep std ≈ 0.6, avoid |tanh| > 0.97

**3. Batch Normalization**
- Forces activations to be Gaussian (mean=0, std=1)
- Makes training more stable and less sensitive to init
- Requires running statistics for inference
- Different behavior in train vs eval mode!

**4. Modular Design**
- Build reusable layer classes
- Stack them like Lego blocks
- This is how PyTorch works!

**5. Training Diagnostics**
- Plot activation distributions
- Plot gradient distributions
- Monitor update/parameter ratios
- Target: ratio ≈ 10⁻³

### Key Takeaways

```python
# Good initialization template
W = torch.randn(fan_in, fan_out) * gain / fan_in**0.5
b = torch.zeros(fan_out)  # or small random

# Monitor these during training:
- activation.mean(), activation.std()
- (activation.abs() > threshold).mean()  # saturation
- (lr * grad).std() / param.std()  # update ratio
```

### Next Steps

1. Try training to convergence
2. Experiment with different architectures (deeper, wider)
3. Try different activation functions (ReLU, GELU)
4. Explore other normalization techniques (LayerNorm, GroupNorm)
5. Move on to RNNs, LSTMs, and Transformers!

**You now understand the foundations of modern deep learning! 🎉**